In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import time
import datetime
import pandas as pd
import numpy as np
from pandas import DataFrame
import requests
import re
import itertools
import urllib.request
import sys, os
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# 스크롤 다운 함수
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.implicitly_wait(3)
        if datetime.datetime.now() > end:
            break

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
searching_keyword_df = pd.read_csv('searching_keyword.csv', encoding='utf-8')
searching_keyword_df = searching_keyword_df.reset_index(drop=True)
searching_keyword_df

In [ ]:
final = []

for idx, row in searching_keyword_df.iterrows():
    
    search = row['검색명']
    print(search)
    # chrome창(웹드라이버) 열기
    driver = webdriver.Chrome("./chromedriver")

    # 네이버 페이지 크롬으로 열기
    url = 'https://m.search.naver.com/search.naver?sm=mtp_hty.top&where=m&query={}'.format(search)
    driver.get(url)

    # 플레이스 페이지 더보기 버튼 누르기
    try:
        driver.implicitly_wait(7)
        sample = driver.find_element_by_xpath('//*[@id="loc-main-section-root"]/div/div[5]/a')
        sample.send_keys('\n')
    except:
        driver.implicitly_wait(7)
        sample = driver.find_element_by_xpath('//*[@id="loc-main-section-root"]/div/div[6]/a')
        sample.send_keys('\n')

    for i in range(1,5+1):

        total_list = []

        # 업체 정보 페이지 들어가기
        driver.implicitly_wait(7)
        sample = driver.find_element_by_xpath('//*[@id="_list_scroll_container"]/div/div/div[2]/ul/li['+str(i)+']/div[2]/a[1]/div/div').click()

        # 업체명
        driver.implicitly_wait(7)
        sample = driver.find_element_by_xpath('//*[@id="_title"]/span[1]')
        name = sample.text

        # 업체 주소
        driver.implicitly_wait(3)
        try:
            sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[4]/div/div[2]/div/ul/li[1]/div/span[1]')
            addr = sample.text
        except:
            sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[4]/div/div[1]/div/ul/li[1]/div/span[1]')
            addr = sample.text

        # 업체 운영시간
        driver.implicitly_wait(3)
        try:
            sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[4]/div/div[2]/div/ul/li[2]/div/div/div')
            time = sample.text
        except:
            try:
                sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[4]/div/div[2]/div/ul/li[2]/div/a/div[1]/div/span')
                time = sample.text
            except:
                sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[4]/div/div[1]/div/ul/li[2]/div/div/div')
                time = sample.text
        
        doScrollDown(15)

        # 업체 키워드
        driver.implicitly_wait(3)
        try:
            sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div/div[2]/div/ul/li[6]/div/ul')
            keyword = sample.text
        except:
            try:
                sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div/div[1]/div/ul/li[6]/div/ul')
                keyword = sample.text
            except:
                sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div/div[1]/div/ul/li[4]/div/ul')
                keyword = sample.text
            

        # 업체 설명
        driver.implicitly_wait(3)
        try:
            sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div/div[2]/div/ul/li[7]/div/a/span[1]')
            explanation = sample.text
        except:
            try:
                sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div/div[1]/div/ul/li[7]/div/a/span[1]')
                explanation = sample.text
            except:
                sample = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div/div[1]/div/ul/li[5]/div/div/span')
                explanation = sample.text
            
        # 업체 사진 탭으로 이동
        driver.implicitly_wait(3)
        sample = driver.find_element_by_xpath('//*[@id="_autoPlayable"]').click()

        # For 문을 이용하여, 페이지 내 이미지 링크 5개 저장
        driver.implicitly_wait(3)
        for j in range(1,5+1):
            try:
                img = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div/div/ul/li['+str(j)+']/a/img')
            except:
                img = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[4]/div/div/ul/li['+str(j)+']/a/img')
            image = img.get_attribute('src')

        total_list = [name, addr, time, keyword, explanation, image]

        final.append(total_list)
        
        driver.back()
        driver.back()

    final = pd.DataFrame(final, columns=['업체명','업체주소','업체운영시간','업체키워드','업체설명','업체사진'])
    final = final.drop_duplicates(['업체명','업체주소','업체운영시간','업체키워드','업체설명','업체사진'], ignore_index = True)
    
    final.to_csv('crawling_'+ search +'.csv', encoding='utf-8')